In [ ]:
!git clone https://github.com/Swastik166/TANS_trial.git
    

In [1]:
import sys
sys.path.insert(0, './TANS_trial')
!pip install config

In [ ]:
%%bash
cd TANS_trial
python3 main.py --gpu 0 --mode train --batch-size 140 --n-epochs 10000 --base-path path/for/storing/outcomes/ --data-path path/to/processed/dataset/is/stored/ --model-zoo path/to/model_zoo.pt --seed 777 

In [45]:
import random
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os
from PIL import Image

batch_size = 32
data_path = '/kaggle/input/ie-643-tans-train-sub-dataset'


'''def get_loader(mode='train'):
    dataset = ZooDatasets(mode=mode)
    root_path = dataset.load_data()
    loader = ImageFolder(root=root_path,transform=transforms.Compose([
                                                  transforms.Resize((224, 224)),
                                                  transforms.ToTensor(),
                                              ]),
                                              )
    loader = DataLoader(dataset=loader,
                        batch_size=batch_size,
                        shuffle=(mode == 'train'),
                        num_workers=4)
    return dataset, loader'''


class ZooDatasets(Dataset):
    
    def __init__(self, mode='train', transform=None):
        #self.args = args
        self.mode = mode
        self.transform = transform
        self.data_path = data_path
        '''self.dataset_list = [
                'bottles',
                'cassava_leaf_disease',
                'casting_products',
                'corals',
                'ct_images',
                'four_shapes',
                'fruits',
                'lego_bricks',
                'natural_images',
                'store_items',
        ]'''
        self.dataset_list = ['corals']
        
        self.curr_dataset = self.dataset_list[0]
        self.load_data()


              
        
    def get_loader(self, mode='train'):
        root_path = self.load_data()
        loader = ImageFolder(root=root_path, transform=transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ]))

        loader = DataLoader(
            dataset=loader,
            batch_size=batch_size,
            shuffle=(mode == 'train'),
            num_workers=4
        )
        
        return loader
        
    def load_data(self):
        if self.mode == 'train':
            self.data_folder = os.path.join(self.data_path, f'{self.curr_dataset}/tr')
        elif self.mode == 'validation':
            self.data_folder = os.path.join(self.data_path, f'{self.curr_dataset}/va')
        elif self.mode == 'test':
            self.data_folder = os.path.join(self.data_path, f'{self.curr_dataset}/te')
            
            
        self.classes = sorted([d.name for d in os.scandir(self.data_folder) if d.is_dir()])
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        self.samples = []
        
        for target_class in self.classes:
            class_index = self.class_to_idx[target_class]
            target_dir = os.path.join(self.data_folder, target_class)
            for root, _, fnames in sorted(os.walk(target_dir, followlinks=True)):
                for fname in sorted(fnames):
                    path = os.path.join(root, fname)
                    # Modify here to convert label to integer
                    label = self.class_to_idx[target_class]
                    self.samples.append((path, label))
        
        
        return self.data_folder
        
     
        
    def get_dataset_list(self):
        return self.dataset_list
    
    
    def curr(self):
        return self.curr_dataset
    
    def set_mode(self, mode):
        self.mode = mode
        
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        path, target = self.samples[index]
        img = Image.open(path).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)
            

        return img, target
    
    
    def set_dataset(self, dataset):
        self.curr_dataset = dataset
        self.load_data()
            
    def get_nclss(self):
        return (len(self.classes))
    
    def get_clss(self):
        return (self.classes)
        

In [ ]:
tr_dataset = ZooDatasets(mode='train')
te_dataset = ZooDatasets(mode='test')
val_dataset = ZooDatasets(mode='validation')

'''intr_loader = intr_dataset.get_loader(mode='train')
inte_loader = inte_dataset.get_loader(mode='test')
inval_loader = inval_dataset.get_loader(mode='validation')'''

print('outer', tr_dataset.curr_dataset)

for query_dataset in tr_dataset.get_dataset_list():
    print('quesry_dataset',query_dataset)
    tr_dataset.set_dataset(query_dataset)
    te_dataset.set_dataset(query_dataset)
    val_dataset.set_dataset(query_dataset)
    
    tr_loader = tr_dataset.get_loader(mode='train')
    te_loader = te_dataset.get_loader(mode='test')
    val_loader = val_dataset.get_loader(mode='validation')
    
    
    print(tr_dataset.curr_dataset)
    x, y = next(iter(tr_loader))
    print('y for tr', y)
    x, y = next(iter(te_loader))
    print('y for te', y)
    
    clss = tr_dataset.get_clss()
    nclss = tr_dataset.get_nclss()
    
    print('nclass', nclss )
    print('clss', clss )
    

    print('tr_loader', len(tr_loader)*batch_size)
    print('te_loader', len(te_loader)*batch_size)
    print('val_loader', len(val_loader)*batch_size)

In [46]:
import torch
import numpy as np
import random
import torchvision.models as models
from tqdm import tqdm
import time
!pip install gdown

"""add utils from misc and use the finctions available in for better error handling"""

noise_path = '/kaggle/working/noise.pt'
learn = 1e-2
patience = 5
n_eps_finetuning = 5

class ModelZoo:
    def __init__(self):
        self.seed = 777
        self.models = {}
        self.train_instances = {}
        self.noise = torch.load(noise_path)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        torch.cuda.manual_seed(self.seed)
        torch.manual_seed(self.seed)
        np.random.seed(self.seed)
        random.seed(self.seed)


        
        
    def init_loaders(self):
        # Get loaders for train, test, and validation data
        print('=====> LOADING dataset and loaders for train, test and validation <=====')
       
        
        self.tr_dataset = ZooDatasets(mode='train')
        self.te_dataset = ZooDatasets(mode='test')
        self.val_dataset = ZooDatasets(mode='validation')
        
        
        
        
        
        
    def create_zoo(self):
        zoo = {}
        self.init_loaders()

            
            
        for query_dataset in self.tr_dataset.get_dataset_list():
            
            
            
            print('quesry_dataset',query_dataset)
            
            self.tr_dataset.set_dataset(query_dataset)
            self.te_dataset.set_dataset(query_dataset)
            self.val_dataset.set_dataset(query_dataset)

            tr_loader = self.tr_dataset.get_loader(mode='train')
            te_loader = self.te_dataset.get_loader(mode='test')
            val_loader = self.val_dataset.get_loader(mode='validation')
            nclass = self.tr_dataset.get_nclss()
            print(nclass)
            
            
            


            for i in range(10):
                # Get neural network model and topology information
                print(f'=====> Generating {i+1}th network  <=====')
                topol, net = self.get_net(nclass)

                # Training the model and obtaining accuracy
                lss = torch.nn.CrossEntropyLoss()
                optim = torch.optim.SGD(net.parameters(), lr=1e-2, momentum=0.9, weight_decay=4e-5)
                scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim , float(n_eps_finetuning))
                
                acc = self.train( net, optim, scheduler, lss, query_dataset, tr_loader, val_loader, nclass, topol)  # Assuming train method returns accuracy

                # Calculating number of parameters
                n_params = self.n_param(net)
                f_emb = self.f_emb(net)

                
                del net
                del optim
                del lss
                
                zoo['dataset'].append(dataset)
                zoo['topol'].append(topol)
                zoo['acc'].append(acc)
                zoo['f_emb'].append(f_emb)
                zoo['n_params'].append(n_params)
                
                x_query_train, x_query_test = self.get_query(tr_loader, te_loader)
                self.save_trainpt(self, dataset, nclss)
                
        self.save_zoo(zoo)
        
                
                


    def get_net(self, nclss):
        super_net_name = "ofa_supernet_mbv3_w12"

        # Load the super network from the 'mit-han-lab/once-for-all' repository
        super_net = torch.hub.load('mit-han-lab/once-for-all', super_net_name, pretrained=True).eval()

        # Sample an active subnet configuration from the super network
        sampled_config = super_net.sample_active_subnet()

        # Extract topology information from the sampled configuration
        pre_topol = list(sampled_config.values())
        topol = []

        for i in pre_topol:
            for j in i:
                topol.append(j)

        # Split the topology into kernel sizes, expansion ratios, and depths
        ks = topol[:20] 
        e = topol[20:40]
        d = topol[40:]

        # Set the active subnet in the super network using the sampled topology
        super_net.set_active_subnet(ks=ks, e=e, d=d)
        active_subnet = super_net.get_active_subnet(preserve_weight=True)
        active_subnet.classifier = torch.nn.Linear(1536, nclss)
        active_subnet = active_subnet.to(self.device)

        return topol, active_subnet

    
    
    
    def f_emb(self, net):
        model = net        
        module = list(model.children())[:-1]
        model = torch.nn.Sequential(*module)
        model.eval()
        with torch.no_grad():
            # Forward pass through ResNet18
            f_emb = model(self.noise)
        print(f_emb)
        return f_emb

    
    
    

    def train(self, model, optim, scheduler, lss, dataset, train_loader, val_loader, nclss, topol):
        #self.model = model
        print(f'Starting Training for:{dataset} with model topology:{topol}')
        lr = learn
        counter = 0
        best_val_loss = 10000
        val_acc = list()
        
        
        for ep in range(n_eps_finetuning):
            curr_ep = ep
            ep_loss_tr = 0.0
            ep_loss_val = 0.0
            ep_tr_time = 0
            st = time.time()
            
            
            model.train()
            for b_id, batch in tqdm(enumerate(train_loader)):
                optim.zero_grad()
                
                x,y = batch
                output = model(x.to(self.device))
                loss = lss(output, y.to(self.device))
                loss.backward()
                optim.step()
                scheduler.step()
                        
                tr_loss = loss.item()
                        
                ep_loss_tr += tr_loss * x.size(0)
                
            ep_loss_tr = ep_loss_tr/len(train_loader)

                        
            model.eval()
            total_val = 0
            crrct_val = 0
                        
                        
            for v_id, (x,y) in tqdm(enumerate(val_loader)):
                outputs = model(x.to(self.device))
                loss_v = lss(outputs, y.to(self.device))
                       
                val_loss = loss_v.detach.item()
                
                ep_loss_val += val_loss * x.size(0)
                        
                pred = torch.argmax(outputs, dim = 1)
                total_val += y.size(0)
                correct_val += (pred == y.to(self.device)).sum().item() 
                        
            acc = (100*correct_val)/total_val
            val_acc.append(acc)
            
            ep_loss_val = ep_loss_val/len(val_loader)
    
            if ep_loss_val < best_val_loss:
                best_val_loss = ep_loss_tr
                counter = 0
                
            elif ep_loss_val > min_loss:
                counter += 1
                
            if counter >= patience:
                print(f"Early stopping on, {ep}th, epoch")
                break
            dura = time.time() -st    
            print(f'ep : {ep}, loss:{ep_loss_tr}, val_loss:{ep_loss_val}, acc:{acc}, time:{dura}')
                
        return val_acc[-1]
            
            
                        
                        
                        
                        
    
    def n_param(self, model):
        # Calculate the number of parameters in the model
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    

    def get_query(self, train_loader, test_loader):
        model = models.resneth18(pretrained = True)
        module = list(model.children())[:-1]
        model = torch.nn.Sequential(*module)
        model.eval()
        with torch.no_grad():
            images, _ =  next(iter(train_loader))
            print(len(images))
            # Forward pass through ResNet18
            query_train = model(images)
        print (query_train.shape)
        with torch.no_grad():
            images, _ =  next(iter(test_loader))
            print(len(images))
            # Forward pass through ResNet18
        query_test = model(images)
        print (query_test.shape)
        return query_train, query_test
    
    
    

    def save_zoo(self, dict):
        # Save model zoo to model_zoo.pt

        torch.save(dict, 'model_zoo.pt')

        
        
    def save_trainpt(self):
        # Save training instances to m_train.pt
        trainpt_dict = {}
        for dataset in self.datasets:
            trainpt_dict[dataset] = {
                'task': {},  # Placeholder for task information
                'clss': {},  # Placeholder for class information
                'nclss': {},  # Placeholder for number of classes
                'x_query_train': {},  # Placeholder for training query data
                'x_query_test': {}  # Placeholder for testing query data
            }
        torch.save(trainpt_dict, 'm_train.pt')


In [47]:
zoo = ModelZoo()
zoo.create_zoo()

=====> LOADING dataset and loaders for train, test and validation <=====
quesry_dataset corals
14
=====> Generating 1th network  <=====


Using cache found in /root/.cache/torch/hub/mit-han-lab_once-for-all_master
Downloading: "https://raw.githubusercontent.com/han-cai/files/master/ofa/ofa_nets/ofa_mbv3_d234_e346_k357_w1.2" to .torch/ofa_nets/ofa_mbv3_d234_e346_k357_w1.2


Starting Training for:corals with model topology:[3, 5, 5, 5, 7, 5, 5, 7, 3, 5, 7, 7, 7, 5, 7, 5, 3, 3, 7, 3, 4, 3, 6, 3, 6, 6, 4, 4, 6, 4, 3, 6, 6, 3, 4, 6, 3, 4, 6, 3, 3, 4, 2, 4, 4]


16it [01:40,  6.27s/it]
0it [00:04, ?it/s]


AttributeError: 'builtin_function_or_method' object has no attribute 'item'

In [ ]:
model_zoo = {'dataset':['dataset1', 'dataset1', 'dataset2', 'dataset2', 'dataset3', 'dataset3'],
             'topol':['model1', 'model2', 'model3', 'model4' , 'model5',  'model6'],
             'acc':['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6'],
             'f_emb':['f_emb1', 'f_emb2', 'f_emb3', 'f_emb4', 'f_emb5', 'f_emb6'],
             'n_params':['np1', 'np2', 'np3', 'np4', 'np5', 'np6']}

In [ ]:
model_zoo

In [ ]:
import torch 
torch.save(model_zoo, 'zoo.pt')

In [ ]:
zoo = torch.load('zoo.pt')
print(zoo)
print(type(zoo))

In [ ]:
zoo['dataset'].append('test')
zoo['topol'].append('test')
zoo['acc'].append('test')
zoo['f_emb'].append('test')
zoo['n_params'].append('test')
print(zoo)

In [5]:
noise = torch.randn((3, 224,224))
torch.save(noise, 'noise.pt')